In [1]:
import re
from collections import defaultdict, Counter

def clean_text(text):
    """Einfache Bereinigung und Tokenisierung."""
    text = text.lower()
    # Nur Buchstaben, Leerzeichen und einige Satzzeichen behalten
    text = re.sub(r'[^a-zäöüß\s.,!?;]', '', text)
    return text

def sent_tokenize(text):
    """Teilt Text in Sätze auf."""
    # Teilen bei Satzzeichen, die auf einen Punkt, Fragezeichen oder Ausrufezeichen folgen
    sentences = re.split(r'[.!?]+', text)
    # Entferne leere Strings aus der Liste
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def word_tokenize(sentence):
    """Teilt Satz in Wörter auf."""
    words = sentence.split()
    return words

def build_ngram_counts(sentences, n=3):
    """Zählt N-Gramm-Häufigkeiten."""
    ngram_counts = defaultdict(lambda: Counter())
    # Füge Start- und End-Symbole hinzu
    start_symbol = '<s>'
    end_symbol = '</s>'

    for sentence in sentences:
        # Füge Start- und End-Symbole hinzu
        words = [start_symbol] * (n - 1) + word_tokenize(sentence) + [end_symbol]
        # Generiere N-Gramme und zähle sie
        for i in range(len(words) - n + 1):
            # Der Kontext ist die Sequenz der ersten n-1 Wörter des N-Gramms
            context = tuple(words[i : i + n - 1])
            # Das nächste Wort ist das letzte Wort des N-Gramms
            next_word = words[i + n - 1]
            ngram_counts[context][next_word] += 1

    return ngram_counts

def predict_next_word_ngram(ngram_counts, context_words, prefix, n=3):
    """
    Sagt die nächsten Wörter basierend auf den Kontextwörtern und einem Präfix voraus.
    """
    # Stelle sicher, dass der Kontext die richtige Länge hat (N-1 Wörter)
    # Wenn weniger als N-1 Wörter verfügbar sind, verwenden wir Start-Symbole
    start_symbol = '<s>'
    if len(context_words) < n - 1:
        # Füge so viele Start-Symbole wie nötig am Anfang hinzu
        context = tuple([start_symbol] * (n - 1 - len(context_words)) + list(context_words))
    else:
        # Nimm die letzten n-1 Wörter als Kontext
        context = tuple(context_words[-(n - 1):])

    suggestions = []
    # Finde alle Wörter, die nach diesem Kontext kommen
    potential_next_words = ngram_counts.get(context, {})

    # Filtere Wörter, die mit dem Präfix beginnen und sortiere nach Häufigkeit
    for next_word, count in potential_next_words.items():
        # Ignoriere das End-Symbol in den Vorschlägen
        if next_word == '</s>':
            continue
        if next_word.startswith(prefix):
            # Berechne die Wahrscheinlichkeit (optional, Zählungen reichen oft für Sortierung)
            # total_count = sum(potential_next_words.values())
            # probability = count / total_count if total_count > 0 else 0
            suggestions.append((next_word, count)) # Speichern als (Wort, Anzahl)

    # Sortiere absteigend nach Häufigkeit
    suggestions.sort(key=lambda item: item[1], reverse=True)

    # Gib nur die Wörter zurück (z.B. Top 5)
    return [word for word, count in suggestions[:5]]

# --- Beispielhafte Nutzung ---

# Ein etwas größerer Beispielkorpus mit Sätzen
corpus = """
Das schnelle Auto fährt die Straße entlang.
Ein Auto ist ein Fahrzeug.
Die Katze sitzt auf dem Sofa.
Das Sofa ist sehr bequem.
Ein Sofa und ein Tisch stehen im Wohnzimmer.
"""

# N-Gramm Ordnung festlegen (z.B. 3 für Trigramme)
N_ORDER = 3

# 1. Korpus vorbereiten und Tokenisieren
cleaned_corpus = clean_text(corpus)
sentences = sent_tokenize(cleaned_corpus)
print(f"Sätze: {sentences}\n")

# 2. N-Gramm-Modell erstellen
ngram_counts = build_ngram_counts(sentences, n=N_ORDER)
# print(f"{N_ORDER}-Gramm Zählungen: {ngram_counts}\n") # Zur Überprüfung

# 3. Auto-Complete simulieren
print(f"--- Auto-Complete Simulation (N={N_ORDER}) ---")

# Beispiel 1: Nach "das schnelle" und Präfix "a"
context = ["das", "schnelle"]
prefix = "a"
vorschlaege = predict_next_word_ngram(ngram_counts, context, prefix, n=N_ORDER)
print(f"Nach '{' '.join(context)}' mit Präfix '{prefix}': {vorschlaege}")
# Erwartung: 'auto'

# Beispiel 2: Nach "ein" und Präfix "f" (Kontext kürzer als N-1, benutzt Start-Symbole)
context = ["ein"]
prefix = "f"
vorschlaege = predict_next_word_ngram(ngram_counts, context, prefix, n=N_ORDER)
print(f"Nach '{' '.join(context)}' mit Präfix '{prefix}': {vorschlaege}")
# Erwartung: 'fahrzeug'

# Beispiel 3: Nach "auf dem" und Präfix "s"
context = ["auf", "dem"]
prefix = "s"
vorschlaege = predict_next_word_ngram(ngram_counts, context, prefix, n=N_ORDER)
print(f"Nach '{' '.join(context)}' mit Präfix '{prefix}': {vorschlaege}")
# Erwartung: 'sofa'

# Beispiel 4: Nach einem Kontext, der nicht im Training vorkam (z.B. "schnelle katze")
context = ["schnelle", "katze"] # Diese Sequenz gibt es im Korpus nicht
prefix = "s"
vorschlaege = predict_next_word_ngram(ngram_counts, context, prefix, n=N_ORDER)
print(f"Nach '{' '.join(context)}' mit Präfix '{prefix}': {vorschlaege}")
# Erwartung: Leere Liste oder Fallback auf niedrigere N-Gramme (nicht in dieser einfachen Impl.)

# Beispiel 5: Am Satzanfang (Kontext enthält Start-Symbole) mit Präfix "d"
context = [] # Leerer Kontext bedeutet Satzanfang
prefix = "d"
vorschlaege = predict_next_word_ngram(ngram_counts, context, prefix, n=N_ORDER)
print(f"Nach '{' '.join(context)}' (Satzanfang) mit Präfix '{prefix}': {vorschlaege}")
# Erwartung: 'das'

Sätze: ['das schnelle auto fährt die straße entlang', 'ein auto ist ein fahrzeug', 'die katze sitzt auf dem sofa', 'das sofa ist sehr bequem', 'ein sofa und ein tisch stehen im wohnzimmer']

--- Auto-Complete Simulation (N=3) ---
Nach 'das schnelle' mit Präfix 'a': ['auto']
Nach 'ein' mit Präfix 'f': []
Nach 'auf dem' mit Präfix 's': ['sofa']
Nach 'schnelle katze' mit Präfix 's': []
Nach '' (Satzanfang) mit Präfix 'd': ['das', 'die']
